In [4]:
import os
import urllib.parse as up
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from config import *

In [5]:
try:
    up.uses_netloc.append("postgres")
    url = up.urlparse(os.environ["DATABASE_URL"])
    conn = psycopg2.connect(
        database=url.path[1:],
        user=url.username,
        password=url.password,
        host=url.hostname,
        port=url.port)
    
    # Create a cursor
    cursor = conn.cursor()
    print("Successfully connected to the database!")
    # You can now execute SQL queries using the 'cursor' object.

except Exception as e:
    print("Unable to connect to the database:", e)


Successfully connected to the database!


### Create schema

In [121]:
# Create the schema
query = "CREATE SCHEMA IF NOT EXISTS recipes_app;"
cursor.execute(query)


### Create table

#### Receitas

In [122]:
query = '''CREATE TABLE recipes (
    id SERIAL PRIMARY KEY,
    name TEXT NOT NULL,
    ingredients TEXT[] NOT NULL,
    preparation_method TEXT[] NOT NULL,
    preparation_time INTERVAL NOT NULL);
'''
cursor.execute(query)

In [130]:
query = "SELECT * FROM recipes"
cursor.execute(query)
print(cursor.fetchall())

[(1, 'Bolo de Chocolate', ['200g de Chocolate', '250g de Farinha'], ['1. Preaqueça o forno a 180°C.', '2. Misture o chocolate e a farinha.', '3. Asse por 30 minutos.'], datetime.timedelta(seconds=3600))]


### Populate table

#### Get recipes

In [123]:
# Dados das receitas
recipes_data = [
    {
        "name": "Bolo de Chocolate",
        "ingredients": ["200g de Chocolate", "250g de Farinha"],
        "preparation_method": ["1. Preaqueça o forno a 180°C.", "2. Misture o chocolate e a farinha.", "3. Asse por 30 minutos."],
        "preparation_time": "1 hour"
    }
]


# Inserir receitas na tabela
for recipe in recipes_data:
    cursor.execute(
        """
        INSERT INTO recipes (name, ingredients, preparation_method, preparation_time)
        VALUES (%s, %s, %s, %s)
        """,
        (recipe["name"],recipe["ingredients"], recipe["preparation_method"], recipe["preparation_time"])
    )


### Commit and close the database connection

In [127]:
conn.commit()
conn.close()

In [ ]:
## Futuro:

import requests
from bs4 import BeautifulSoup
from psycopg2.extras import Json


# Função para extrair informações de uma receita
def extract_recipe_info(recipe_url):
    response = requests.get(recipe_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Título da receita
    title_element = soup.find('header', class_='u-title-page')
    print(title_element)
    if title_element is not None:
        title = title_element.text.strip()
    else:
        title = "Título não encontrado"

    # Ingredientes da receita
    ingredients = []
    ingredients_elements = soup.find_all('span', class_='recipe-ingredients-item-label')
    for ingredient_element in ingredients_elements:
        ingredient_name_element = ingredient_element.find('span', class_='recipe-ingredients-item-label')
        if ingredient_name_element is not None:
            ingredient_name = ingredient_name_element.text.strip()
            ingredients.append({"name": ingredient_name})

    # Método de preparo
    preparation_method_element = soup.find('li', class_='recipe-steps-item')
    if preparation_method_element is not None:
        preparation_method = preparation_method_element.find('div', class_='recipe-steps-text').text.strip()
    else:
        preparation_method = "Método de preparo não encontrado"

    # Tempo de preparo
    preparation_time_element = soup.find('time', datetime=True)
    if preparation_time_element is not None:
        preparation_time = preparation_time_element['title']
    else:
        preparation_time = "Tempo de preparo não encontrado"

    return {
        "title": title,
        "ingredients": ingredients,
        "preparation_method": preparation_method,
        "preparation_time": preparation_time
    }


# Lista de URLs de receitas
recipe_urls = [
    "https://www.tudogostoso.com.br/receita/318901-macarrao-pomodoro.html",
    "https://www.tudogostoso.com.br/receita/309-lasanha-de-berinjela.html"
    # Insira mais URLs de receitas aqui...
]

# Extrair informações de cada receita e inserir no banco de dados
for recipe_url in recipe_urls:
    recipe_info = extract_recipe_info(recipe_url)
    cursor.execute(
        """
        INSERT INTO recipes (name, ingredients, preparation_method, preparation_time)
        VALUES (%s, %s, %s, %s)
        """,
        (recipe_info["name"], Json(recipe_info["ingredients"]), recipe_info["preparation_method"], recipe_info["preparation_time"])
    )